# [NAPARI](https://napari.org/)
## and [cellpose](https://cellpose-napari.readthedocs.io/en/latest/index.html) (installer [here](https://pypi.org/project/cellpose-napari/0.1.3/))

To upgrade run in a cell:
```python
!pip install cellpose-napari==0.1.3
!pip install cellpose --upgrade
```

```bash
napari -w cellpose-napari
```

### Basic usage:
```python
viewer = napari.view_image(data.astronaut(), rgb=True)
nbscreenshot(viewer)
viewer.close()
```

example notebooks [here](https://github.com/sofroniewn/napari-training-course/blob/master/lessons/)

In [ ]:
## to install
# !pip install napari[all]
# !pip install QT
# !pip install napari[pyqt5]
# !pip install cellpose-napari==0.1.4
# !pip install napari-nikon-nd2
# !pip install magicgui
# !pip install napari-pyclesperanto-assistant
# !pip install napari-clusters-plotter #hdbscan failed
# !pip install napari-plot-profile
# !pip install napari-brightness-contrast
# !pip install napari-curtain
# !pip install napari-3d-ortho-viewer
# !pip install napari-manual-split-and-merge-labels
# !pip install napari-crop
# !pip install napari-stl-exporter

# !conda install -c conda-forge ipywidgets

## to upgrade:
# !pip install cellpose-napari==0.1.4
# !pip install cellpose-napari --upgrade
# !pip install cellpose --upgrade
# !pip install napari --upgrade
# !pip install mxnet-mkl

In [ ]:
import napari
from magicgui import magicgui
from enum import Enum
import cellpose_napari
import cellpose
from cellpose import models
from napari.utils import nbscreenshot
from tifffile import imread
import numpy as np
from scipy import ndimage
from scipy.stats import mannwhitneyu
import napari_nikon_nd2
import os 
from magicgui import magicgui
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import importlib
import svgutils
from svgutils.compose import *
import ipywidgets
import time
import sys
from plotParams import *


pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Dark',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
pBG

In [ ]:
def applyPlotStyle(plotStyleString):
    if plotStyleString=='Dark':
        # dark background
        params = {"ytick.color" : "w",
                  "xtick.color" : "w",
                  "axes.labelcolor" : "w",
                  "axes.edgecolor" : "w",
                 "axes.linewidth" : 3,
                 "xtick.major.width" : 3,
                 "ytick.major.width" : 3,
                 "xtick.major.size" : 8,
                 "ytick.major.size" : 8,
                 "text.color" : "w"}
        plt.rcParams.update(params)
        plt.style.use('dark_background')
    elif plotStyleString=='Light':
        # white background
        params = {"ytick.color" : "k",
                  "xtick.color" : "k",
                  "axes.labelcolor" : "k",
                  "axes.edgecolor" : "k",
                 "axes.linewidth" : 3,
                 "xtick.major.width" : 3,
                 "ytick.major.width" : 3,
                 "xtick.major.size" : 8,
                 "ytick.major.size" : 8,
                 "text.color" : "k"}
    plt.rcParams.update(params)
    font_prop = font_manager.FontProperties(fname='/System/Library/Fonts/Avenir.ttc')
    matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=batlow)
    print('Plotting style is ' + plotStyleString)
applyPlotStyle(pBGMenu.value)

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.close()

***
# Index <a id='Index'></a>
***
- [Extraction of z planes for analysis](#zExtract)
- [Segmentation with cellpose](#cellSeg)
- [Manual correction of segmentation](#manualCuration)
- [Create thumbnails _WIP_](#thumbnails)
- [Quantification](#quantification)

***
## Extract layers from z-stacks<a id='zExtract' name='zExtract'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# getting list of files
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20220121_nr2F0/"
gene = 'nr2'
#### !ls $dPath/*$gene*[002,004].nd2
# fList = !ls $dPath/*$gene*[002,004].nd2
# fList = list(map(lambda st: str.replace(st, dPath + '/', 'fPath = \"'), fList))
# fList = list(map(lambda st: str.replace(st, '.nd2', '\"; gene = \'' + gene + '\'; '), fList))
# print(*fList, sep = "\n")

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"

# open file

# F3[tbx2a]
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20220121_nr2F0/"
fPath = "20220121_5dpf_xOG_aCT_wt_L01_002"; gene = 'wt'; zlims = [30,34]; # ok
fPath = "20220121_5dpf_xOG_aCT_wt_L02_002"; gene = 'wt'; zlims = [33,37];
fPath = "20220121_5dpf_xOG_aCT_wt_L02_004"; gene = 'wt'; zlims = [39,42];
fPath = "20220121_5dpf_xOG_aCT_wt_L03_002"; gene = 'wt'; zlims = [38,41];
# #### fPath = "20220121_5dpf_xOG_aCT_wt_L03_004"; gene = 'wt'; zlims = [58,62]; # side-view. Excluding
fPath = "20220121_5dpf_xOG_aCT_wt_L04_002"; gene = 'wt'; zlims = [33,36];
fPath = "20220121_5dpf_xOG_aCT_wt_L05_002"; gene = 'wt'; zlims = [36,39];
fPath = "20220121_5dpf_xOG_aCT_wt_L06_002"; gene = 'wt'; zlims = [24,28];
fPath = "20220121_5dpf_xOG_aCT_wt_L07_002"; gene = 'wt'; zlims = [26,29];
fPath = "20220121_5dpf_xOG_aCT_wt_L07_004"; gene = 'wt'; zlims = [30,36];
fPath = "20220121_5dpf_xOG_aCT_wt_L08_002"; gene = 'wt'; zlims = [34,39];
fPath = "20220121_5dpf_xOG_aCT_wt_L08_004"; gene = 'wt'; zlims = [36,52]; #curved; deep mip
fPath = "20220121_5dpf_xOG_aCT_wt_L09_002"; gene = 'wt'; zlims = [31,37]; # dorsal rotation. EXCLUDE
# # ####fPath = "20220121_5dpf_xOG_aCT_wt_L09_004"; gene = 'wt'; zlims = [45,48]; # could not avoid ONH. exclude
# fPath = "20220121_5dpf_xOG_aCT_wt_L10_002"; gene = 'wt'; zlims = [33,37];
# fPath = "20220121_5dpf_xOG_aCT_wt_L10_004"; gene = 'wt'; zlims = [28,33];
# # ####fPath = "20220121_5dpf_xOG_aCT_wt_L11_002"; gene = 'wt'; zlims = [23,26]; # dorsal rotation. Exclude
# fPath = "20220121_5dpf_xOG_aCT_wt_L11_004"; gene = 'wt'; zlims = [25,28];
# fPath = "20220121_5dpf_xOG_aCT_wt_L12_002"; gene = 'wt'; zlims = [23,29];
# fPath = "20220121_5dpf_xOG_aCT_wt_L13_002"; gene = 'wt'; zlims = [37,40];
# fPath = "20220121_5dpf_xOG_aCT_wt_L13_004"; gene = 'wt'; zlims = [31,34];
# # ####fPath = "20220121_5dpf_xOG_aCT_wt_L14_002"; gene = 'wt'; zlims = [43,49]; # dorsal rotation. Exclude
# fPath = "20220121_5dpf_xOG_aCT_wt_L15_002"; gene = 'wt'; zlims = [31,37];
# fPath = "20220121_5dpf_xOG_aCT_wt_L16_002"; gene = 'wt'; zlims = [35,40];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L01_002"; gene = 'nr2e3'; zlims = [32,36];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L01_004"; gene = 'nr2e3'; zlims = [30,34];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L02_002"; gene = 'nr2e3'; zlims = [26,32];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L02_004"; gene = 'nr2e3'; zlims = [26,30];
# # ####fPath = "20220121_5dpf_xOG_aCT_nr2e3_L03_002"; gene = 'nr2e3'; zlims = [35,40]; # dorsal rotation. Exclude
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L03_004"; gene = 'nr2e3'; zlims = [28,32];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L05_002"; gene = 'nr2e3'; zlims = [19,21];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L06_002"; gene = 'nr2e3'; zlims = [28,32]; # EXAMPLE: no rods in ventral patch!
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L06_004"; gene = 'nr2e3'; zlims = [33,38];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L07_002"; gene = 'nr2e3'; zlims = [31,37];
# # ####fPath = "20220121_5dpf_xOG_aCT_nr2e3_L07_004"; gene = 'nr2e3'; zlims = [11,27]; #side view, ventral patch. 
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L08_002"; gene = 'nr2e3'; zlims = [24,27];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L08_004"; gene = 'nr2e3'; zlims = [23,27];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L09_002"; gene = 'nr2e3'; zlims = [25,28];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L09_004"; gene = 'nr2e3'; zlims = [18,24]; # 0 rods in FOV
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L10_002"; gene = 'nr2e3'; zlims = [46,50];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L10_004"; gene = 'nr2e3'; zlims = [25,28];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L11_002"; gene = 'nr2e3'; zlims = [6,18]; # curved; bad for DAPI
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L12_002"; gene = 'nr2e3'; zlims = [20,25]; # EXAMPLE for extreme nalor
# # ####fPath = "20220121_5dpf_xOG_aCT_nr2e3_L12_004"; gene = 'nr2e3'; zlims = [28,31]; # dorsal rotation; EXCLUDE
# # ####fPath = "20220121_5dpf_xOG_aCT_nr2e3_L14_002"; gene = 'nr2e3'; zlims = [24,28]; # dorsal rotation; EXCLUDE
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L15_002"; gene = 'nr2e3'; zlims = [22,27];
# # ####fPath = "20220121_5dpf_xOG_aCT_nr2e3_L15_004"; gene = 'nr2e3'; zlims = [17,23]; # dorsal rotation; EXCLUDE
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L16_002"; gene = 'nr2e3'; zlims = [14,19];
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L16_004"; gene = 'nr2e3'; zlims = [29,35]; # no rods in FOV
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L01_002"; gene = 'nr2f1b'; zlims = [43,46];
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L01_004"; gene = 'nr2f1b'; zlims = [30, 34];
# # ####fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L02_002"; gene = 'nr2f1b'; zlims = [31,35]; # dorsal rotation. EXCLUDE
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L02_004"; gene = 'nr2f1b'; zlims = [29,35];
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L03_002"; gene = 'nr2f1b'; zlims = [26,31];
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L03_004"; gene = 'nr2f1b'; zlims = [32,39];
# # ####fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L04_002"; gene = 'nr2f1b'; zlims = [19,23]; # extreme dorsal rotation. EXCLUDE
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L04_004"; gene = 'nr2f1b'; zlims = [32,36];
# # ####fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L05_002"; gene = 'nr2f1b'; zlims = [22,29]; # dorsal rotation. EXCLUDE
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L05_004"; gene = 'nr2f1b'; zlims = [20,24];
# # ####fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L06_002"; gene = 'nr2f1b'; zlims = [20,24]; # dorsal rotation. EXCLUDE
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L07_002"; gene = 'nr2f1b'; zlims = [22,25]; # borderline dorsal
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L08_002"; gene = 'nr2f1b'; zlims = [31,37]; # curvy
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L08_004"; gene = 'nr2f1b'; zlims = [27,31];
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L09_002"; gene = 'nr2f1b'; zlims = [27,31];
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L09_004"; gene = 'nr2f1b'; zlims = [27,31];
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L10_002"; gene = 'nr2f1b'; zlims = [25,28]; # very good DAPI. Try segmentation of nuclei?
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L10_004"; gene = 'nr2f1b'; zlims = [27,31];
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L11_002"; gene = 'nr2f1b'; zlims = [37,41]; # ventral rotation, but probably still acceptable?
# fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L11_004"; gene = 'nr2f1b'; zlims = [20,24];
# # ####fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L12_002"; gene = 'nr2f1b'; zlims = [19,23]; # dorsal rotation. EXCLUDE

# create subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
if (os.path.isdir(dOut)==False):
    os.mkdir(dOut)
    print('Created output directory')

print('Canvas clear')

In [ ]:
# load whole stack to figure out best planes for cellpose
print('Loading files...')
viewer.open((dPath+fPath + ".nd2"))
viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();

# if len(viewer.layers)==2: # no DAPI, GFP and RFP
#     nChannels = 2
#     viewer.layers[0].colormap = 'green'
#     viewer.layers[0].name = 'R'
#     viewer.layers[1].colormap = 'magenta'
#     viewer.layers[1].name = 'U'
if len(viewer.layers)==2: # DAPI and RFP channel (single positive larvae with bad immunos)
    nChannels = 2
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 0.66
    viewer.layers[1].colormap = 'magenta'
    viewer.layers[1].name = 'C'
elif len(viewer.layers)==3: #DAPI
    nChannels = 3
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 0.33
    viewer.layers[1].colormap = 'green'
    viewer.layers[1].name = 'R'
    viewer.layers[2].colormap = 'magenta'
    viewer.layers[2].name = 'C'
elif len(viewer.layers)==4: #DAPI and immunos
    nChannels = 4
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 0.33
    viewer.layers[1].colormap = 'green'
    viewer.layers[1].name = 'rho'
    viewer.layers[2].colormap = 'magenta'
    viewer.layers[2].name = 'C'
    viewer.layers[3].colormap = 'bop orange'
    viewer.layers[3].name = 'sws1'

print('Loaded: ' + fPath)

In [ ]:
# make mips (and remove any previous ones)
# # code fr eyes without immuno
# zlims = [19,23];
if len(viewer.layers)>nChannels:
    for l in viewer.layers[nChannels:]:
        viewer.layers.remove(l)

# for l in [viewer.layers[0],viewer.layers[1],viewer.layers[2],viewer.layers[3]]: # save all channels including immunos
for l in [viewer.layers[0],viewer.layers[1],viewer.layers[2]]: # DAPI and FP channels
# for l in [viewer.layers['N'],viewer.layers['U']]: #DAPI and RFP
# for l in [viewer.layers['N'],viewer.layers['U'],viewer.layers['R']]: #DAPI and RFP and GPF
    l.visible = False
#     viewer.layers.remove(l.name + '_mip')
    viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")
    
    
# # for FPs ------- # for OSs # code for eyes with 1D4 immuno
# zlims = [28,30]; zlims2 = [14,18];
# if len(viewer.layers)>3:
#     for l in viewer.layers[3:]:
#         viewer.layers.remove(l)

# for l in [viewer.layers[0],viewer.layers[1]]:
#     l.visible = False
# #     viewer.layers.remove(l.name + '_mip')
#     viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")
    
# for l in [viewer.layers[2]]:
#     l.visible = False
# #     viewer.layers.remove(l.name + '_mip')
#     viewer.add_image(l.data[zlims2[0]:zlims2[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

In [ ]:
# when things look good, save in folder for batch cellpose analysis
l = viewer.layers['R_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['C_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['N_mip']; l.save(dOut + l.name + '.tiff')
print('MIP layers saved for ' + fPath)

### Establish landmarks in whole-eye MIP
- Use eyeMarks layer to mark:
        - Landmark #1: Center of FOV (where zoom in is centered)
        - Landmark #2: Center of Optic Nerve Head 
        - Landmark #3: Dorsal edge of eye
        - Landmark #4: Ventral edge of eye
        - Landmark #5: Temporal edge of eye (strike zone side)
        - Landmark #6: Nasal edge of eye
- Decide if stack should be excluded (and stop analysis here and make note in csv summary file)

In [ ]:
nMIP = 3 * 2
viewer.layers['C_mip'].scale = [1.5/8,1.5/8]; viewer.layers['C_mip'].translate = [0,+(512-1024*1.5/(8*2))]
viewer.layers['R_mip'].scale = [1.5/8,1.5/8]; viewer.layers['R_mip'].translate = [0,+(512-1024*1.5/(8*2))]
viewer.layers['N_mip'].scale = [1.5/8,1.5/8]; viewer.layers['N_mip'].translate = [0,+(512-1024*1.5/(8*2))]
viewer.layers['N_mip'].visible = False;

# Load whole-eye MIP to establish landmarks
print('Loading eye mip...')
if str.endswith(fPath,'002'):
    mipPath = fPath[:-1] + "1_mip";
elif str.endswith(fPath,'004'):
    mipPath = fPath[:-1] + "3_mip";
viewer.open((dPath+mipPath + ".nd2"))

viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();

if len(viewer.layers)==2+nMIP: # no DAPI, GFP and RFP
    nChannels = 2
    viewer.layers[nMIP].colormap = 'green'
    viewer.layers[nMIP].name = 'eyeR'
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeC'
if len(viewer.layers)==2+nMIP: # DAPI and RFP channel (single positive larvae with bad immunos)
    nChannels = 2
    viewer.layers[nMIP+0].colormap = 'green'
    viewer.layers[nMIP+0].name = 'eyeR'
    viewer.layers[nMIP+0].opacity = 0.66
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeC'
elif len(viewer.layers)==3+nMIP: #DAPI
    nChannels = 3
    viewer.layers[nMIP+0].colormap = 'gray'
    viewer.layers[nMIP+0].name = 'eyeN'
    viewer.layers[nMIP+0].opacity = 0.33
    viewer.layers[nMIP+1].colormap = 'green'
    viewer.layers[nMIP+1].name = 'eyeR'
    viewer.layers[nMIP+2].colormap = 'magenta'
    viewer.layers[nMIP+2].name = 'eyeC'


print('Loaded: ' + fPath)

eyeMarks = viewer.add_points(size=20, name = 'eyeMarks', symbol="cross", face_color='#ffffff80')
viewer.layers.select_next();

# Save eye landmarks
eyeMarksButton = ipywidgets.Button(description='Save eyeMarks', layout=ipywidgets.Layout(width='200px', height='100px'))
out = ipywidgets.Output()
def eyeMClick(_):
    with out:
        l = viewer.layers['eyeMarks']; l.save(dOut + l.name + '.csv');
        print('eyeMarks layers saved for ' + fPath)
    # clear viewer
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    eyeMarksButton.disabled = True # to prevent double clicking warnings. All sales are final!
    
eyeMarksButton.on_click(eyeMClick)

print("Make 6 landmark points then click button to save")

ipywidgets.VBox([eyeMarksButton,out])

### [Go back and run next one &uarr;](#zExtract)
---

***
## Segmentation using cellpose<a id='cellSeg'>∮</a>
***
[Back to Index](#Index)

## Run cellpose as batch analysis
Usually good parameters are :
1. xOPS:GFP: diameter = 20; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws1:mCherry: diameter = 40; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws2:mCherry:
1. mws2:GFP:
1. thrb:tdTomato:
    
flow_threshold = 0.6 will give more ROIs

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

In [ ]:
# define analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"

### collect file paths
# gene = 'wt';
# fPaths = [
# 	'20220121_5dpf_xOG_aCT_wt_L01_002',
# 	'20220121_5dpf_xOG_aCT_wt_L02_002',
# 	'20220121_5dpf_xOG_aCT_wt_L02_004',
# 	'20220121_5dpf_xOG_aCT_wt_L03_002',
# 	'20220121_5dpf_xOG_aCT_wt_L03_004',
# 	'20220121_5dpf_xOG_aCT_wt_L04_002',
# 	'20220121_5dpf_xOG_aCT_wt_L05_002',
# 	'20220121_5dpf_xOG_aCT_wt_L06_002',
# 	'20220121_5dpf_xOG_aCT_wt_L07_002',
# 	'20220121_5dpf_xOG_aCT_wt_L07_004',
# 	'20220121_5dpf_xOG_aCT_wt_L08_002',
# 	'20220121_5dpf_xOG_aCT_wt_L08_004',
# 	'20220121_5dpf_xOG_aCT_wt_L09_002',
# 	'20220121_5dpf_xOG_aCT_wt_L10_002',
# 	'20220121_5dpf_xOG_aCT_wt_L10_004',
# 	'20220121_5dpf_xOG_aCT_wt_L11_002',
# 	'20220121_5dpf_xOG_aCT_wt_L11_004',
# 	'20220121_5dpf_xOG_aCT_wt_L12_002',
# 	'20220121_5dpf_xOG_aCT_wt_L13_002',
# 	'20220121_5dpf_xOG_aCT_wt_L13_004',
# 	'20220121_5dpf_xOG_aCT_wt_L15_002',
	# '20220121_5dpf_xOG_aCT_wt_L16_002',
# ]

### collect file paths
# gene = 'nr2e3';
# fPaths = [
# 	'20220121_5dpf_xOG_aCT_nr2e3_L01_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L01_004',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L02_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L02_004',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L03_004',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L05_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L06_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L06_004',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L07_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L07_004',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L08_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L08_004',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L09_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L09_004',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L10_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L10_004',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L11_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L12_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L15_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L16_002',
# 	'20220121_5dpf_xOG_aCT_nr2e3_L16_004',
# ]

### collect file paths
# gene = 'nr2f1b';
# fPaths = [
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L01_002',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L01_004',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L02_004',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L03_002',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L03_004',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L04_004',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L05_002',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L05_004',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L07_002',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L08_002',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L08_004',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L09_002',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L09_004',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L10_002',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L10_004',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L11_002',
# 	'20220121_5dpf_xOG_aCT_nr2f1b_L11_004',
# ]


# define cellpose params
cpParams = {
    'model' : 'cyto2', # default is 'cyto' or 'cyto2'
    'net_avg' : True,
    'channels' : [0,0], #single channel without nucleus info
    'diameterR' : 25,
    'diameterC' : 40,
    'flow_threshold' : 0.6,
    'cellprob_threshold' : 0.0
}

# define model to use (e.g. 'cyto2')
model = models.Cellpose(gpu=False, model_type=cpParams['model'])

In [ ]:
print('Starting analysis:\n')
with tqdm(total=len(fPaths), file=sys.stdout) as progBar:
    for fPath in fPaths:
        print('\n' + fPath)
        print('\t Rods')
        startTime = time.time()
        dOut = dAnalysis + gene + '/' + fPath + '/' + 'R_mip.tiff'
        img = cellpose.io.imread(dOut)
    #     imgplot = plt.imshow(img)
        masks, flows, styles, diams = model.eval(img, 
                                                 diameter=cpParams['diameterR'], channels=cpParams['channels'],
                                                 do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                 flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
        cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
        endTime = time.time()
        print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
        progBar.update(1)
    
#     print('\t UV cones')
#     dOut = dAnalysis + gene + '/' + fPath + '/' + 'U_mip.tiff'
#     img = cellpose.io.imread(dOut)
# #     imgplot = plt.imshow(img)
#     masks, flows, styles, diams = model.eval(img, 
#                                              diameter=cpParams['diameterC'], channels=cpParams['channels'],
#                                              do_3D=False, net_avg = cpParams['net_avg'], interp = True,
#                                              flow_threshold = cpParams['flow_threshold'], mask_threshold = cpParams['mask_threshold'])
#     cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
    
print('\nFinished cellpose batch analysis')

***
## Manual correction of cellpose segmentation<a id='manualCuration'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# viewer = napari.Viewer()

In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"

# open file
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20220121_nr2F0/"

fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L01_002"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L01_004"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L02_004"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L03_002"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L03_004"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L04_004"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L05_004"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L07_002"; gene = 'nr2f1b'; # borderline dorsal
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L08_002"; gene = 'nr2f1b'; # curvy
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L08_004"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L09_002"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L09_004"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L10_002"; gene = 'nr2f1b'; # very good DAPI. Try segmentation of nuclei?
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L10_004"; gene = 'nr2f1b';
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L11_002"; gene = 'nr2f1b'; # ventral rotation, but probably still acceptable?
fPath = "20220121_5dpf_xOG_aCT_nr2f1b_L11_004"; gene = 'nr2f1b';

# fPath = "20220121_5dpf_xOG_aCT_wt_L01_002"; gene = 'wt'; # ok
# fPath = "20220121_5dpf_xOG_aCT_wt_L02_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L02_004"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L03_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L04_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L05_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L06_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L07_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L07_004"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L08_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L08_004"; gene = 'wt'; #curved; deep mip
# fPath = "20220121_5dpf_xOG_aCT_wt_L10_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L10_004"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L11_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L11_004"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L12_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L13_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L13_004"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L15_002"; gene = 'wt';
# fPath = "20220121_5dpf_xOG_aCT_wt_L16_002"; gene = 'wt';

# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L01_002"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L01_004"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L02_002"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L02_004"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L03_004"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L05_002"; gene = 'nr2e3'; # EXAMPLE for DAPI
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L06_002"; gene = 'nr2e3'; # EXAMPLE: no rods in ventral patch!
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L06_004"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L07_002"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L08_002"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L08_004"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L09_002"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L09_004"; gene = 'nr2e3'; # 0 rods in FOV
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L10_002"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L10_004"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L11_002"; gene = 'nr2e3'; # curved; bad for DAPI
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L12_002"; gene = 'nr2e3'; # EXAMPLE for extreme nalor
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L15_002"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L16_002"; gene = 'nr2e3';
# fPath = "20220121_5dpf_xOG_aCT_nr2e3_L16_004"; gene = 'nr2e3'; # no rods in FOV

# define subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
print('Viewer cleared...')

# clear key binds
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    ...

# load mips
mipO = 0.65
viewer.open(dOut + 'R' + "_mip.tiff", plugin='builtins', colormap = 'gray', blending='additive', opacity=mipO);
viewer.open(dOut + 'C' + "_mip.tiff", plugin='builtins', colormap = 'magenta', blending='additive', opacity=mipO);
viewer.open(dOut + 'N' + "_mip.tiff", plugin='builtins', colormap = 'bop blue', blending='additive', opacity=mipO);

# load segmentation
segData = np.load(dOut + 'R' + "_mip" + "_seg.npy", allow_pickle=True).item()
viewer.add_labels(segData['masks'], name='R_seg',blending='additive');
viewer.layers['R_seg'].preserve_labels = True;
# tweak display to help with segmentation
viewer.layers['R_mip'].opacity=0.40;viewer.layers['R_mip'].gamma=0.7;viewer.layers['R_mip'].contrast_limits=[150,2000]

# segData = np.load(dOut + 'U' + "_mip" + "_seg.npy", allow_pickle=True).item()
# viewer.add_labels(segData['masks'], name='U_seg',blending='additive');
# viewer.layers['U_seg'].visible = False
# viewer.layers['U_seg'].preserve_labels = True;


viewer.layers['C_mip'].visible = False;
viewer.layers['N_mip'].visible = False;
# viewer.layers['R_seg'].contour = 5
# viewer.layers['U_seg'].contour = 5

print('Loaded  ' + fPath + ' !\n Now fix it and save it!')

#define useful keyboard shortcuts

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = 'R_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')

print('removeLabel on R_seg ("Shift-X")')

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = 'R_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle R_seg preserve_labels ("k")')
        
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = 'C_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle C_mip visibility ("B")')

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
    lname = 'R_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle R_mip visibility ("N")')

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
    """Set the currently selected label to the largest used label plus one."""
    lname = 'R_seg'
    viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in R_seg visibility ("+"|"M")')

---

#### resave curated segmentation after napari-ing around (only doing R_mip)

In [ ]:
baseName = 'R'
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + fPath)

# If NUCS mip revealed UV cones
baseName = 'U'
lname = baseName + '_missing'
l = viewer.layers['Points']; l.save(dOut + baseName + '_missing');
print("Image has {0} UV-cone nuclei".format(len(viewer.layers['Points'].data)))

### Reload curated segmentations after saving

In [ ]:
# remove all except mips
n_mips = 3;
if len(viewer.layers)>n_mips:
    for l in viewer.layers[n_mips:]:
        viewer.layers.remove(l)

viewer.open(dOut + 'R' + "_seg_curated.tiff", name='R_seg', plugin='builtins');
# # viewer.layers['R_seg'].contour = 5
# # viewer.layers['R_mip'].visible = True
# viewer.open(dOut + 'U' + "_seg_curated.tiff", name='U_seg', plugin='builtins');

print(fPath)
nR = len(np.unique(viewer.layers['R_seg'].data))-1
print("Image has {0} Rods".format(nR))
# nU = len(np.unique(viewer.layers['U_seg'].data))-1
# print("Image has {0} UV cones".format(nU))

### [Go back and run next one &uarr;](#manualCuration)
---

### reload mip and eye marks in case there are doubts

In [ ]:

nMIP = 3 + 1
viewer.layers['R_seg'].visible = False;
viewer.layers['R_mip'].visible = False;
viewer.layers['N_mip'].visible = False;
viewer.layers['C_mip'].visible = False;

# Load whole-eye MIP to establish landmarks
print('Loading eye mip...')
if str.endswith(fPath,'002'):
    mipPath = fPath[:-1] + "1_mip";
elif str.endswith(fPath,'004'):
    mipPath = fPath[:-1] + "3_mip";
viewer.open((dPath+mipPath + ".nd2"))

viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();

if len(viewer.layers)==2+nMIP: # no DAPI, GFP and RFP
    nChannels = 2
    viewer.layers[nMIP].colormap = 'green'
    viewer.layers[nMIP].name = 'eyeR'
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeC'
if len(viewer.layers)==2+nMIP: # DAPI and RFP channel (single positive larvae with bad immunos)
    nChannels = 2
    viewer.layers[nMIP+0].colormap = 'green'
    viewer.layers[nMIP+0].name = 'eyeR'
    viewer.layers[nMIP+0].opacity = 0.66
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeC'
elif len(viewer.layers)==3+nMIP: #DAPI
    nChannels = 3
    viewer.layers[nMIP+0].colormap = 'gray'
    viewer.layers[nMIP+0].name = 'eyeN'
    viewer.layers[nMIP+0].opacity = 0.33
    viewer.layers[nMIP+1].colormap = 'green'
    viewer.layers[nMIP+1].name = 'eyeR'
    viewer.layers[nMIP+2].colormap = 'magenta'
    viewer.layers[nMIP+2].name = 'eyeC'

eyeMarksData  = np.loadtxt(dOut + 'eyeMarks.csv',delimiter=',',skiprows=1,usecols=[1,2])
eyeMarks = viewer.add_points(eyeMarksData,size=20, name = 'eyeMarks', symbol="cross", face_color='#ffffff80')

In [ ]:
# # manual merging of labels
# mergeLabels = [167,168]
# # lname = 'M_seg'
# lname = 'L_seg'
# if viewer.layers[lname].visible == False:
#     print('Layer not active')
# else:
#     tempd = viewer.layers[lname].data
#     tempd[tempd == mergeLabels[1]]=mergeLabels[0]
#     viewer.layers[lname].data = tempd
#     print('Merged in ' + lname)

In [ ]:
# # manually delete label
# selLabel = 49
# lname = 'L_seg'
# tempd = viewer.layers[lname].data
# tempd[tempd == selLabel]=0
# viewer.layers[lname].data = tempd

# for selLabel in np.arange(7,100):
#     lname = 'U_seg'
#     tempd = viewer.layers[lname].data
#     tempd[tempd == selLabel]=0
#     viewer.layers[lname].data = tempd

***
## Quantification<a id='quantification'></a>
***
[Back to Index](#Index)

### Read csv created during analysis and create bar plot

In [ ]:
#gene Colors
zfC = {
    'R'  : '#7d7d7d',
    'U' : '#B73AB9',
    'S' : '#4364F6',
    'M' : '#59CB3B',
    'L' : '#CE2A22',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'nr2e3' : '#7d7d7d',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'nr2e3' : '+',
    
}

prLabel = {
    'R'  : 'Rods',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}


def formatFigureMain(figH, axH, plotH):
#     font_path = 'C:/Users/pataklk/Documents/Frag_Analysis_Code/Avenir.ttc'
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30) # was 18
    fontLabels = font_manager.FontProperties(fname=font_path, size=36) # was 22
    fontTitle = font_manager.FontProperties(fname=font_path, size=30) # was 28 
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def formatFigure(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
#     axH.set_xlabel('wt vs. cr', fontproperties=fontLabels)
    axH.set_ylabel('cells per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

def formatFigureRvU(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('Rods per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('UV cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)
    
def formatFigureMvS(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('M cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('S cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

    
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

# dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/tbx2bF3s_CRnr2e3F0s/xOGs1C/"
# fName = "tbx2bF3s_CRnr2e3F0s_counts_Updated.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','nr2e3','nr2f1b']
nGenes = np.size(geneList)
# photoreceptors = ['R','U','S','M','L']
photoreceptors = ['R']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['gene']==gene) & (df['excludeFlag']==False)][pr]
        jitter = np.random.randn(len(countData))*0.01
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg")